In [70]:
import pandas as pd
import os
import re
import codecs
import requests
from bs4 import BeautifulSoup
    
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

In [47]:
def modify_link(link):
    '''
        Function to modify the link format
    '''
    # Split the link using '?' and take the first part (before the query parameters)
    base_url = link.split('?')[0]
    
    # Split the base_url using '/' and return the second last part
    return base_url.split('/')[-1]

In [48]:
def text_file_to_dataframe(filename):
    # Check if the text file exists
    if os.path.exists(filename):
        
        # Read the text file and create a list of lines
        with open(filename, "r") as file:
            lines = file.readlines()
            content = [line.strip() for line in lines]

        # Create a DataFrame from the list
        df = pd.DataFrame(content, columns=['links'])
        
        return df
    else:
        print(f"{filename} does not exist.")
        return None

In [49]:
def prepare_link_df(df):
    # Apply the modify_link function to the 'data' column
    df['links'] = df['links'].apply(modify_link)
        
    return df

In [50]:
def save_dataframe_to_text_file(df, filename):
    # Save the modified DataFrame to a text file with newline-separated values
    df.to_csv(filename, index=False, header=False, sep='\n')

In [51]:
def get_reviews(df, num_loops):
    # initiate selenium browser
    driver = webdriver.Chrome()
    review_1_show_more_button_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[2]/div/button/span[1]'   
    review_1_show_more_button = driver.find_element_by_xpath(review_1_show_more_button_tag)
    review_1_body_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[1]/span'
    review_1_body = driver.find_element_by_xpath(review_1_body_tag)
    
    # iterate through every row in DataFrame column 'links'
    for i in df['links']:
        # go to starter link on browser
        driver.get(i)
        review_1_show_more_button.click()
        print(review_1_body.text)

In [58]:
def get_reviews(df, num_loops):
    # initiate selenium browser
    driver = webdriver.Chrome()

    # iterate through the dataframe for num_loops times
    for i in range(num_loops):
        # variable definitions
        link = df['links'].iloc[i]
        
        # go to starter link on browser
        driver.get(link)
        
        # wait 10 seconds
        review_1_show_more_button_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[2]/div/button/span[1]'   
        # extract button element
        review_1_show_more_button = driver.find_element_by_xpath(review_1_show_more_button_tag)
        
        # click on show more button
        review_1_show_more_button.click()
        
        # wait 10 seconds
        review_1_body_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[1]/span'
        
        # extract review
        review_1_body = driver.find_element_by_xpath(review_1_body_tag)
        
        # S H O W
        print(review_1_body.text)
            
    # close the browser
    driver.quit()

In [74]:
def get_reviews_3(df, num_loops):
    # initiate selenium browser
    driver = webdriver.Chrome()

    # iterate through the dataframe for num_loops times
    for i in range(num_loops):
        # variable definitions
        link = df['links'].iloc[i]
        
        # go to starter link on browser
        driver.get(link)
        
        # wait 10 seconds for show more button to appear and click on it
        try:
            review_1_show_more_button_tag = '//*[@id="ReviewsSection"]/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[2]/div/button'
            review_1_show_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, review_1_show_more_button_tag)))
            review_1_show_more_button.click()
        except (TimeoutException, NoSuchElementException):
            print(f"Could not find show more button for link: {link}")
            continue
        
        # wait 10 seconds for review to appear and extract it
        try:
            review_1_body_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[1]/span'
            review_1_body = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, review_1_body_tag)))
            print(review_1_body.text)
        except (TimeoutException, NoSuchElementException):
            print(f"Could not find review for link: {link}")
            continue
            
    # close the browser
    driver.quit()

In [122]:
def get_reviews_4(df, num_loops):
    # initiate selenium browser
    driver = webdriver.Chrome()

    # iterate through the dataframe for num_loops times
    for i in range(num_loops):
        # variable definitions
        link = df['links'].iloc[i]
        
        # go to starter link on browser
        driver.get(link)
        
        for j in range(5):
            review_body_tag_dictionary = {'review':['//*[@id="ReviewsSection"]/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[1]/span',
                            '//*[@id="ReviewsSection"]/div[5]/div[2]/div[3]/article/section/section[2]/section/div/div[1]/span',
                            '//*[@id="ReviewsSection"]/div[5]/div[2]/div[4]/article/section/section[2]/section/div/div[1]/span',
                            '//*[@id="ReviewsSection"]/div[5]/div[2]/div[5]/article/section/section[2]/section/div/div[1]/span',
                            '//*[@id="ReviewsSection"]/div[5]/div[2]/div[6]/article/section/section[2]/section/div/div[1]/span'           
                            ]}
            # wait 10 seconds for review to appear and extract it
            try:
                review_1_body_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[1]/span'
                review_1_body = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, review_1_body_tag)))
                print(f"Book {i+1}: ")
                print(f"Review {j+1}: ")
                print(review_1_body.text)
                print("\n")
            except (TimeoutException, NoSuchElementException):
                print(f"Could not get review searching for \"more button\": {link}")
                print("\n")
                continue

            # wait 10 seconds for show more button to appear and click on it
            try:
                review_1_show_more_button_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[2]/div/button/'
                review_1_show_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, review_1_show_more_button_tag)))
                review_1_show_more_button.click()
            except (TimeoutException, NoSuchElementException):
                print(f"Could not find show more button for link: {link}")
                print("\n")
                continue
            except ElementClickInterceptedException:
                print(f"Could not click on show more button for link: {link}")
                print("\n")
                continue

            # wait 10 seconds for review to appear and extract it
            try:
                review_1_body_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[1]/span'
                review_1_body = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, review_1_body_tag)))
                print(f"Book {i+1}: ")
                print(f"Review {j+1}: ")
                print(review_1_body.text)
                print("\n")
            except (TimeoutException, NoSuchElementException):
                print(f"Could not find review for link: {link}")
                print("\n")
                continue
            
    # close the browser
    driver.quit()

# Main

In [123]:
input_filename = "links.txt"
df = text_file_to_dataframe(input_filename)

In [ ]:
get_reviews_4(df,5)

# Manually Testing

In [80]:
# initiate selenium browser
driver = webdriver.Chrome()

In [83]:
link = 'https://www.goodreads.com/book/show/13771831-breaking-news?from_search=true&from_srp=true&qid=mipV5pB2vA&rank=1'

In [84]:
# go to starter link on browser
driver.get(link)

# Click on Link

In [85]:
review_1_show_more_button_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[2]/div/button/'

In [ ]:
review_1_show_more_button = driver.find_element_by_xpath(review_1_show_more_button_tag)

# If Link Doesn't Exists

In [ ]:
review_body_tag_dictionary = {}

In [100]:
review_1_body_tag = '//*[@id="ReviewsSection"]/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[1]/span'

In [101]:
review_2_body_tag = '//*[@id="ReviewsSection"]/div[5]/div[2]/div[3]/article/section/section[2]/section/div/div[1]/span'

In [102]:
review_3_body_tag = '//*[@id="ReviewsSection"]/div[5]/div[2]/div[4]/article/section/section[2]/section/div/div[1]/span'

In [103]:
review_4_body_tag = '//*[@id="ReviewsSection"]/div[5]/div[2]/div[5]/article/section/section[2]/section/div/div[1]/span'

In [104]:
review_5_body_tag = '//*[@id="ReviewsSection"]/div[5]/div[2]/div[6]/article/section/section[2]/section/div/div[1]/span'

In [89]:
review_1_body = driver.find_element_by_xpath(review_1_body_tag)

In [92]:
review_2_body = driver.find_element_by_xpath(review_2_body_tag)

In [111]:
review_3_body = driver.find_element_by_xpath(review_3_body_tag)

In [112]:
review_4_body = driver.find_element_by_xpath(review_4_body_tag)

In [105]:
review_5_body = driver.find_element_by_xpath(review_5_body_tag)

In [108]:
print(review_1_body.text)

Not the greatest until about the last third of the book. Maybe my taste in authors is changing.


In [107]:
print(review_2_body.text)

This is the fifth sequel in this series. A little better than the last couple. The Godmothers are funny but the story line needs to end. I have the sixth book on my bookshelf hope it's better.


In [113]:
print(review_3_body.text)

This book reminded me of The Golden Girls television series. It included some of the more recent references to people and events that have occured since the series.


In [114]:
print(review_4_body.text)

fern michaels never fails to provide a story that you just cannot put down, I cant wait for her next one


In [106]:
print(review_5_body.text)

Another good read!
